In [ ]:
import numpy as np
import glob
import os
import astropy.constants as const

I wrote a separate script to load in the planet data so we don't have to deal with that here. It returns a dictionary, with the planet names as keys, and the contents of the csv file as a Numpy array

In [ ]:
from load_planet_data import load

In [ ]:
xv_data = load()

In [ ]:
xv_data['Pluto'].shape

Next I will compute the semimajor axis and eccentricityj values of Pluto using the different coding styles.

# Procedural
In this paradigm, we write all the steps needed to accomplish our task in order.

### 1. Compute the solar gravitational constant in the appropriate units.

In [ ]:
AU2M = const.au.value
GMSunSI = const.GM_sun.value
MSun = const.M_sun.value
JD2S = 86400

GMSunAUD = GMSunSI * JD2S**2 / AU2M**3

### 2. Compute the 2-body Gravitational parameter $\mu=G\left(M_{sun}+M_{pluto}\right)$.

In [ ]:
Mpluto = 1.0 / 1.35e8
mu = GMSunAUD * (1 + Mpluto)

### 3. Extract the position and velocity vectors for Pluto.

Here we have to be mindful with how data is stored. I wrote the code for reading in data in a way thatThe data is a 2D array, where the time values in column 1, the components of the position vector in columns 2-4 and velocity vector in columns 5-7. The rows are the values at each point in time.

In [ ]:
tvals = xv_data['Pluto'][:,0]
rvec = xv_data['Pluto'][:,1:4]
vvec = xv_data['Pluto'][:,4:7]

A very common issue is to get the axes mixed up when working with Numpy arrays, so it's always good to verify the shapes. I have 1001 time values and 3 dimensions, so I should end up with shapes of `(1001,3)` for my cartesian vectors and `(1001,)` for my time vector.

In [ ]:
tvals.shape, rvec.shape, vvec.shape

### 4. Compute the magnitudes of the position, velocity, and angular momentum

For this, I'll use Numpy's `norm` method from the linear algebra package. This lets me compute the row-wise magnitude of an (N,3) cartesian vector

In [ ]:
rmag = np.linalg.norm(rvec,axis=1)
vmag = np.linalg.norm(vvec,axis=1)

Verify that I did it correctly by checking the shape

In [ ]:
rmag.shape, vmag.shape

Same procedure with the angular momentum

In [ ]:
h = np.cross(rvec,vvec)

In [ ]:
h.shape

In [ ]:
hmag = np.linalg.norm(h,axis=1)
hmag.shape

### 5. Compute semimajor axis and eccentricity values

Working with Numpy arrays allows us to perform basic arithmetic operations on whole arrays (add, subtract, multiply, divide, raise to a power, etc.). Numpy will *broadcast* the operation along all the rows, which is why we always must be mindful that our shapes make sense. The arrays `rmag`, `vmag`, and `hmag` are all `(1001,)` arrays (1-D), and so we should be able to broadcast our operations without having to iterate over the rows.

In [ ]:
a = 1.0/(2.0 / rmag - vmag**2/mu)
ecc = np.sqrt(1 - hmag**2 / (mu * a))

Verify that this makes sense by checking the values against what we expect. From Table A.2 in the Murray & Dermott book, Pluto's semimajor axis and eccentricity values are ~39.5 and ~0.25. If we've done our operations correctly, our answers should be close to that (though unlikely to be exactly the same, as the values do change over time).

In [ ]:
print(f"Pluto's semimajor axis at t0: {a[0]:.1f} AU")
print(f"Pluto's eccentricity at t0: {ecc[0]:.2f}")

# Functional
In this paradigm, we write each individual step as a function, then apply the functions in order to compute the final results. We will make sure that all of our functions are *pure* (or at least as pure as Python allows us to be, as it's not strictly a functional programming language). To do this, we need to make sure that the functions *always* return the same values for the same arguments, and so are free of side effects. The functions cannot change anything about the state of the program: data is passed in via arguments, is not allowed to change, and new data is passed back upon return.

### 1. Compute the solar gravitational constant in the appropriate units.

In [ ]:
AU2M = const.au.value
GMSunSI = const.GM_sun.value
MSun = const.M_sun.value
JD2S = 86400

GMSunAUD = GMSunSI * JD2S**2 / AU2M**3

### 2. Compute the 2-body Gravitational parameter $\mu=G\left(M_{sun}+M_{pluto}\right)$.

In [ ]:
Mpluto = 1.0 / 1.35e8
mu = GMSunAUD * (1 + Mpluto)

### 3. Extract the position and velocity vectors for Pluto.

In [ ]:
tvals = xv_data['Pluto'][:,0]
rvec = xv_data['Pluto'][:,1:4]
vvec = xv_data['Pluto'][:,4:7]

### 4. Compute the magnitudes of the position, velocity, and angular momentum

In [ ]:
rmag = np.linalg.norm(rvec,axis=1)
vmag = np.linalg.norm(vvec,axis=1)

In [ ]:
h = np.cross(rvec,vvec)

In [ ]:
hmag = np.linalg.norm(h,axis=1)

### 5. Compute semimajor axis and eccentricity values

In [ ]:
a = 1.0/(2.0 / rmag - vmag**2/mu)
ecc = np.sqrt(1 - hmag**2 / (mu * a))

In [ ]:
print(f"Pluto's semimajor axis at t0: {a[0]:.1f} AU")
print(f"Pluto's eccentricity at t0: {ecc[0]:.2f}")

# Object-Oriented
In this paradigm, we encapsulate our data and the functions that act on the data into a construct known as an *object*. The structure of the object is defined using a `Class` definition. Unlike Functional programming, where communication of the state of the program is strictly limited to argument passing, in object-oriented programming the *methods* of the object (that is, the functions that are defined as a part of the object's class) can (and usually do) modify the state of the the object. 

In [ ]:
class Orbits:
    def __init__(self, your_own_initialization_arguments):
        return